In [1]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup, SoupStrainer
import httplib2
import re
import pandas as pd
import math

In [2]:
baselink = "https://ca.indeed.com"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"}

In [3]:
df = pd.DataFrame(columns=['Title', 'Location', 'Extract_date', 'Link', 'Desc', 
                           'Company', 'CompanyLink', 'Salary', 'Job_Type',
                           'Remote', 'Reviews', 'Stars'])

In [4]:
# Generate url from position, location and date posted.
def get_url(position, location, dateposted):
    template = 'https://ca.indeed.com/jobs?q={}&l={}&fromage={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    dateposted = dateposted.replace(' ', '+')
    url = template.format(position, location, dateposted)
    return url
url = get_url('data analyst', 'canada', '1')
print(url)

https://ca.indeed.com/jobs?q=data+analyst&l=canada&fromage=1


In [5]:
r = requests.get(url, headers = headers)
soup = BeautifulSoup(r.content, 'html.parser')
pages = soup.find(id="searchCountPages").get_text().strip()
pages = pages.replace("Page 1 of ", "")
pages = pages.replace(" jobs", "")
X = (int(math.ceil(int(pages)/15)))*10
print(X)

110


In [6]:
for number in range(0, X, 10):
    links = url+'&start={}'.format(number)
    r = requests.get(links, headers = headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    jobcards = soup.find("div", attrs={"id": "mosaic-zone-jobcards"})
    links = []
    
    for l in jobcards.find_all('a', attrs={'href': re.compile("vjs=3")}):
        links.append(l.get('href'))
        
    for n in links:
        newlink = baselink + str(n)
        r = requests.get(newlink, headers = headers)
        soup = BeautifulSoup(r.text, 'html.parser')
        extract_date = datetime.today().strftime('%Y-%m-%d')
        loc = soup.find(class_="icl-u-xs-mt--xs icl-u-textColor--secondary jobsearch-JobInfoHeader-subtitle jobsearch-DesktopStickyContainer-subtitle")
        com = soup.find(class_="jobsearch-CompanyInfoContainer")
        sal = soup.find(class_="jobsearch-JobMetadataHeader-item")
        rat = soup.find(class_="icl-u-lg-block icl-u-lg-mr--sm icl-u-xs-hide")
        
        try:
            reviews = rat.find('div', string=re.compile("reviews")).text.strip()
        except AttributeError:
            reviews = "NA"
                        
        try:
            stars = rat.select_one('meta[content]')['content']
        except AttributeError:
            stars = "NA"
                        
        try:
            desc = soup.find(class_="jobsearch-jobDescriptionText").text.strip()
        except AttributeError:
            desc = "NA"
                        
        try:
            job_title = soup.find(class_="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title").text.strip()
        except AttributeError:
            job_title = "NA"
                        
        try:
            job_type = sal.find('span', string=re.compile("Full|Part|Casu|Intern|Perman|Contr|term|Temp|Freel|Apprent")).text.strip()
        except AttributeError:
            job_type = "NA"
            
        try:
            remote = loc.find('div', string=re.compile("Remote|remote")).text.strip()
        except AttributeError:
            remote = "NA"
                        
        try:
            salary = sal.find('span', string=re.compile("year|hour")).text.strip()
        except AttributeError:
            salary = "NA"
                        
        try:
            companylink = com.find('a', attrs={'href': re.compile("//")}).get('href')
        except AttributeError:
            companylink = "NA"
                        
        try:
            company = com.find('a', attrs={'href': re.compile("//")}).string.strip()
        except AttributeError:
            company = "NA"
                        
        try:
            location = loc.find('div', string=re.compile("ON|SK|BC|AB|PE|NS|MB|QC|NL|PE|YT|NB|NT|NU")).string.strip()
        except AttributeError:
            location = "NA"
                        
        df.loc[len(df.index)] = [job_title, location, extract_date,  newlink, desc, company, companylink, salary, job_type, remote, reviews, stars]

In [9]:
date = datetime.today().strftime('%Y-%m-%d')
fileName = "indeedjobs" + date + ".csv" 
print(fileName)

indeedjobs2022-04-09.csv


In [10]:
df.to_csv(fileName, index = False)